# nn-UNet v2 Tutorial
Masoom Haider, MD, FRCPC, FSAR, FCAHS  
Director AIMS Lab, Sinai Health System  
Joint Dept of Medical Imaging, Univeristy of Toronto  
Oct 18, 2025
\
\
GitHub Library for nnUNet  
https://github.com/MIC-DKFZ/nnUNet  
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring 
method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.

This example uses some of the data from PI-CAI  
https://pi-cai.grand-challenge.org/

## Pre-requisites
- Familiarity with Python coding - intermediate to expert level
- Jupyter notebook installed in python environment


## Installation/Requirements
- See https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/installation_instructions.md  
- Will work on a single GPU workstation/gaming system i.e NVidia 4090 24G VRAM for abdo CT and MRI ($3-5K) 2025
- GPU support for pytorch install


In [ ]:
# install nnUNetv2
# %pip install nnunetv2 #uncomment this line to install

## Preparing Data
https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/dataset_format.md  
Preparing data is use case specific and is not fully covered here

### DICOM to NIfTI
- Many open source AI computer vision models don't directly support DICOM
- Use the NIfTI format (Neuroimaging Informatics Technology Initiative)based requiring DICOM to NIfTI conversion
- Watch out for series naming, mutiphase in one series, patched image volumes
- Preparing and checking series can be time consuming and can benefit from AI tools for series extraction (research)
- Watch out
  - data compression schemes
  - volume concantenation (multiphase, broken between chest , abdomen, pelvis)
  - LAS-DICOM
  - RAI (NIfTI)
  - supine vs prone
  - embedded localizers

**Python Libraries or Viewer Conversion - Not covered in this example**
- **SimpleITK**
- nibabel https://nipy.org/nibabel/
- dicom2nifti https://github.com/icometrix/dicom2nifti
- 3D Slicer https://www.slicer.org/


#### CLEANUP PRIOR RUNS-OPTIONAL
YOU MAY NEED TO RESTART THIS NOTEBOOK or the NOTEBOOK SERVER BEFORE RUNNING THIS CELL

In [1]:
# CLEANUP PRIOR DATA - OPTIONAL - DO THIS IF YOU HAVE RUN THIS BEFORE AND WANT TO START AGAIN
import os, shutil

scratchdir = os.path.join('MLDATADIR','NNUNETTUTORIAL', 'NNUNET')
if os.path.exists(scratchdir):
    shutil.rmtree(scratchdir)


### Setup Directory Structure and Environment Variables used by nnU-Net

In [2]:
import os
from utils.nnunetutils import fInitialize

mldatadir = os.path.join('MLDATADIR','NNUNETTUTORIAL')  # replace with your ML data path in this variable
os.makedirs(mldatadir,exist_ok=True)

rootdir = os.path.join(mldatadir, 'NNUNET')
modelname = 'Dataset101_prostate'
dirdic = fInitialize(rootdir, modelname)  # creates directory structure and returns dictionary of dir locations


ENVIRONMENT VARIABLES
nnUNet_raw:MLDATADIR\NNUNETTUTORIAL\NNUNET\nnUNetv2_raw
nnUNet_preprocessed:MLDATADIR\NNUNETTUTORIAL\NNUNET\nnUNetv2_preprocessed
nnUNet_results:MLDATADIR\NNUNETTUTORIAL\NNUNET\nnUNetv2_results


### Sorting Data and Folder Structure
Data and models are stored in a directory structure defined my some environment variables which must be setup before training and prediction  
Same convention is used for training and prediction

- MLDIRNAME
  - nnUNet_raw
      - Dataset001_BrainTumour *(this is the name of the model)*
      - Dataset002_Heart
      - Dataset003_Liver
      - Dataset004_Hippocampus
      - Dataset005_Prostate
        - dataset.json
        - imagesTr
          - I_001_0000.nii.gz *(channel 0 = axt2)*
          - I_001_0001.nii.gz *(channel 1 = adc)*
          - I_001_0002.nii.gz *(channel 2 = b1600)*
          - ...
        - imagesTs
        - labelsTr *(one label=segmentatotin masks regardless of number of channels)*
          - I_001.nii.gz
          - I_002.nii.gz
          - ...

#### Copy files to the nnUNetv2_raw directory in the format required by nnU-Net

In [3]:
from utils.nnunetutils import fPrepareFilesMP

# where the PICAI data is located
sourcedir = os.path.join(mldatadir, "PICAI")

labels = {
    'background': 0,
    'tumor': 1
}

region_class_order = [1]
imagefilenames = ["axt2.nii.gz", "adc-axt2.nii.gz","b1600-axt2.nii.gz"]
segfilename = "t1_axt2_he.nii.gz"
channel_names = {0: "axt2", 1:"adc", 2:"b1600"}  

fPrepareFilesMP(dirdic, channel_names, labels, sourcedir, imagefilenames, segfilename, region_class_order)


A total of 5 of 5 are for training
Copy MLDATADIR\NNUNETTUTORIAL\PICAI\AIPR_10012_20120802_1000012\t1_axt2_he.nii.gz to MLDATADIR\NNUNETTUTORIAL\NNUNET\nnUNetv2_raw\Dataset101_prostate\labelsTr\IM_001.nii.gz
1/5: MLDATADIR\NNUNETTUTORIAL\PICAI\AIPR_10012_20120802_1000012\axt2.nii.gz
Copy MLDATADIR\NNUNETTUTORIAL\PICAI\AIPR_10012_20120802_1000012\axt2.nii.gz to MLDATADIR\NNUNETTUTORIAL\NNUNET\nnUNetv2_raw\Dataset101_prostate\imagesTr\IM_001_0000.nii.gz
1/5: MLDATADIR\NNUNETTUTORIAL\PICAI\AIPR_10012_20120802_1000012\adc-axt2.nii.gz
Copy MLDATADIR\NNUNETTUTORIAL\PICAI\AIPR_10012_20120802_1000012\adc-axt2.nii.gz to MLDATADIR\NNUNETTUTORIAL\NNUNET\nnUNetv2_raw\Dataset101_prostate\imagesTr\IM_001_0001.nii.gz
1/5: MLDATADIR\NNUNETTUTORIAL\PICAI\AIPR_10012_20120802_1000012\b1600-axt2.nii.gz
Copy MLDATADIR\NNUNETTUTORIAL\PICAI\AIPR_10012_20120802_1000012\b1600-axt2.nii.gz to MLDATADIR\NNUNETTUTORIAL\NNUNET\nnUNetv2_raw\Dataset101_prostate\imagesTr\IM_001_0002.nii.gz
Copy MLDATADIR\NNUNETTUTORIA

## Preprocess the Images
Determine imaging parameters to create an optimal UNet model architecture  
This can take a while as all files are read and metrics tabulated like spatial resolution, segmentation sizes etc.  
https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/assets/nnU-Net_overview.png

![alt text](image.png)

In [4]:
from utils.nnunetutils import fPreProcess
fPreProcess(modelname)

Executing command line: nnUNetv2_plan_and_preprocess -d 101 --verify_dataset_integrity


## Train the Model
This will take 30min for 50 epochs on a Nvidia 4090 machine for one fold

In [6]:
from utils.nnunetutils import fTrain
configurations = ['3d_fullres'] #['2d', '3d_fullres', '3d_lowres', '3d_cascade_fullres']
folds = [1]
fTrain(modelname, configurations=configurations, folds=folds, nepochs=10)

**** Using Trainer 1 ****
Setting CUDA_VISIBLE_DEVICES to 0
Executing command line: nnUNetv2_train Dataset101_prostate 3d_fullres 1 -tr nnUNetTrainer_10epochs


## Predict Using Model
This cell repeats some intialization code so it can be run on its own

In [ ]:
import os
from utils.nnunetutils import fPredict, fInitialize

mldatadir = os.path.join('MLDATADIR','NNUNETTUTORIAL')
rootdir = os.path.join(mldatadir, 'NNUNET')
modelname = 'Dataset101_prostate'
dirdic = fInitialize(rootdir, modelname)  # creates directory structure and returns dictionary of dir locations

configurations = '3d_fullres'
folds = 1

# folder with case you wish to predict
mFolder = os.path.join(mldatadir,'PICAITEST', r'AIPR_10005_20120718_1000005')  
filestopredict  = ["axt2.nii.gz", "adc-axt2.nii.gz","b1600-axt2.nii.gz"]
sourcefnames = [os.path.join(mFolder, f) for f in filestopredict]
segfilename = os.path.join(mFolder,"t1_axt2_nnunet-101.nii.gz")

fPredict(rootdir, modelname, sourcefnames, segfilename, configurations, folds, nepochs=10)

nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.

ENVIRONMENT VARIABLES
nnUNet_raw:MLDATADIR\NNUNETTUTORIAL\NNUNET\nnUNetv2_raw
nnUNet_preprocessed:MLDATADIR\NNUNETTUTORIAL\NNUNET\nnUNetv2_preprocessed
nnUNet_results:MLDATADIR\NNUNETTUTORIAL\NNUNET\nnUNetv2_results
**** Using Trainer 1 ****
copying MLDATADIR\NNUNETTUT